# Bottle Processing
Author: Andrew Reed

### Motivation:
Independent verification of the suite of physical and chemical observations provided by OOI are critical for the observations to be of use for scientifically valid investigations. Consequently, CTD casts and Niskin water samples are made during deployment and recovery of OOI platforms, vehicles, and instrumentation. The water samples are subsequently analyzed by independent labs for  comparison with the OOI telemetered and recovered data.

However, currently the water sample data routinely collected and analyzed as part of the OOI program are not available in a standardized format which maps the different chemical analyses to the physical measurements taken at bottle closure. Our aim is to make these physical and chemical analyses of collected water samples available to the end-user in a standardized format for easy comprehension and use, while maintaining the source data files. 

### Approach:
Generating a summary of the water sample analyses involves preprocessing and concatenating multiple data sources, and accurately matching samples with each other. To do this, I first preprocess the ctd casts to generate bottle (.btl) files using the SeaBird vendor software following the SOP available on Alfresco. 

Next, the bottle files are parsed using python code and the data renamed following SeaBird's naming guide. This creates a series of individual cast summary (.sum) files. These files are then loaded into pandas dataframes, appended to each other, and exported as a csv file containing all of the bottle data in a single data file.

### Data Sources/Software:

* **sbe_name_map**: This is a spreadsheet which maps the short names generated by the SeaBird SBE DataProcessing Software to the associated full names. The name mapping originates from SeaBird's SBE DataProcessing support documentation.

* **Alfresco**: The Alfresco CMS for OOI at alfresco.oceanobservatories.org is the source of the ctd hex, xmlcon, and psa files necessary for generating the bottle files needed to create the sample summary sheet.

* **SBEDataProcessing-Win32**: SeaBird vendor software for processing the raw ctd files and generating the .btl files.


In [1]:
# Import packages used in this notebook
import os, sys, re
import pandas as pd
import numpy as np

In [2]:
# Load the name mapping for the column names
# Specifiy the local directory
sbe_name_map = pd.read_excel('/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')
# sbe_name_map = pd.read_excel('C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')

In [163]:
basepath = '/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/'
array = 'Pioneer/'
cruise = 'Pioneer-05/'
water_dir = 'Water Sampling/'
ctd_dir = 'ctd/'

In [52]:
# Specify the local directory where the bottle (.btl) files are stored for a particular cruise
# dirpath = 'C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/ctd/'
btlpath = basepath+array+cruise+ctd_dir
summary_sheet_path = basepath+array+cruise+water_dir+'Pioneer2_KN-217_sampling_log-1.xlsx'
salts_and_o2_path = basepath+array+cruise+water_dir+'Salts and O2/'
nutrients_path = basepath+array+cruise+water_dir+'Pioneer II Nutrient Data 2014.xlsx'
chl_path = basepath+array+cruise+water_dir+'CHLs KN-214.xls'

In [ ]:
#os.listdir(btlpath)

In [ ]:
# Parse the data for the start_time
def parse_header(header):
    """
    Parse the header of bottle (.btl) files to get critical information
    for the summary spreadsheet.
    
    Args:
        header - an object containing the header of the bottle file as a list of
            strings, split at the newline.
    Returns:
        hdr - a dictionary object containing the start_time, filename, latitude,
            longitude, and cruise id.
    """
    hdr = {}
    for line in header:
        if 'start_time' in line.lower():
            start_time = pd.to_datetime(re.split('= |\[',line)[1])
            hdr.update({'Start Time':start_time.strftime('%Y-%m-%dT%H:%M:%SZ')})
        elif 'filename' in line.lower():
            hex_name = re.split('=',line)[1].strip()
            hdr.update({'Hex name':hex_name})
        elif 'latitude' in line.lower():
            start_lat = re.split('=',line)[1].strip()
            hdr.update({'Start Latitude':start_lat})
        elif 'longitude' in line.lower():
            start_lon = re.split('=',line)[1].strip()
            hdr.update({'Start Longitude':start_lon})
        elif 'cruise id' in line.lower():
            cruise_id = re.split(':',line)[1].strip()
            hdr.update({'Cruise ID':cruise_id})
        else:
            pass
    
    return hdr
        

In [ ]:
# Now write a function to autopopulate the bottle summary sample sheet
files = [x for x in os.listdir(btlpath) if '.btl' in x]
for filename in files:
    filepath = os.path.abspath(btlpath+filename)
    
    # Load the raw content into memory
    with open(filepath) as file:
        content = file.readlines()
    content = [x.strip() for x in content]
    
    # Now parse the file content
    header = []
    columns = []
    data = []
    for line in content:
        if line.startswith('*') or line.startswith('#'):
            header.append(line)
        else:
            try:
                float(line[0])
                data.append(line)
            except:
                columns.append(line)
    
    # Parse the header
    hdr = parse_header(header)
    
    # Parse the column identifiers
    column_dict = {}
    for line in columns:
        for i,x in enumerate(line.split()):
            try:
                column_dict[i] = column_dict[i] + ' ' + x
            except:
                column_dict.update({i:x})
    
    # Parse the bottle data based on the column header locations
    data_dict = {x:[] for x in column_dict.keys()}

    for line in data:
        if line.endswith('(avg)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            for i,x in enumerate(values):
                data_dict[i].append(x)
        elif line.endswith('(sdev)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            data_dict[1].append(values[0])
        else:
            pass
            
    data_dict[1] = [' '.join(item) for item in zip(data_dict[1][::2],data_dict[1][1::2])]
    
    # With the parsed data and column names, match up the data and column
    # based on the location
    results = {}
    for key,item in column_dict.items():
        values = data_dict[key]
        results.update({item:values})
        
    # Put the results into a dataframe
    df = pd.DataFrame.from_dict(results)
        
    # Now add the parsed info from the header files into the dataframe
    for key,item in hdr.items():
        df[key] = item
        
    # Get the cast number
    cast = filename[filename.index('.')-3:filename.index('.')]
    df['Cast #'] = str(cast).zfill(3)
    
    # Generate a filename for the summary file
    outname = filename.split('.')[0] + '.sum'
    
    # Save the results
    df.to_csv(btlpath+outname)

In [ ]:
# Now, for each "summary" file, load and append to each other
df = pd.DataFrame()
for file in os.listdir(btlpath):
    if '.sum' in file:
        df = df.append(pd.read_csv(btlpath+file))
    else:
        pass

In [ ]:
sbe_name_map['Short Name'].apply(lambda x: str(x).lower());

In [ ]:
# Rename the column title using the sbe_name_mapping 
for colname in list(df.columns.values):
    try:
        fullname = list(sbe_name_map[sbe_name_map['Short Name'].apply(lambda x: str(x).lower() == colname.lower()) == True]['Full Name'])[0]
        df.rename({colname:fullname},axis='columns',inplace=True)
    except:
        pass

In [ ]:
df.rename(columns={'Bottle Position':'Niskin #'},inplace=True)
df['Niskin #'] = df['Niskin #'].apply(lambda x: str( int(x) ) )
df.drop(columns='Unnamed: 0',inplace=True)
df['Cast #'] = df['Cast #'].apply(lambda x: str(x).zfill(3) )

In [ ]:
df

In [ ]:
df.to_csv(btlpath+'CTD_Summary.csv')

### Oxygen & Salinity 
Now, we need to add the 

In [ ]:
def clean_sal_files(dirpath):
    
    # Run check if files are held in excel format or csvs
    csv_flag = any(files.endswith('.SAL') for files in os.listdir(dirpath))
    if csv_flag:
        for files in os.listdir(dirpath):
            sample = []
            salinity = []
            with open(basepath+array+cruise+'Water Sampling/Salts and O2/'+leg+files) as file:
                data = file.readlines()
                for ind1,line in enumerate(data):
                    if ind1 == 0:
                        strs = data[0].replace('"','').split(',')
                        cruisename = strs[0]
                        station = strs[1]
                        cast = strs[2]
                        case = strs[8]
                    elif int(line.split()[0]) == 0:
                        pass
                    else:
                        strs = line.split()
                        sample.append(strs[0])
                        salinity.append(strs[2]) 
                # Generate a pandas dataframe to populate data
                data_dict = {'Cruise ID':cruisename,'Station #':station,'Cast #':cast,'Case':case,'Sample ID':sample,'Salinity [psu]':salinity}
                df = pd.DataFrame.from_dict(data_dict)
                df.to_csv(file.name.replace('.','')+'.csv')
    
    else:
        # If the files are already in excel spreadsheets, they've been cleaned into a
        # logical tabular format
        pass
    

def process_sal_files(dirpath):
    
    # Check if the files are excel files or not
    excel_flag = any(files.endswith('SAL.xlsx') for files in os.listdir(dirpath))
    # Initialize a dataframe for processing the salinity files
    df = pd.DataFrame()
    if excel_flag:
        for file in os.listdir(dirpath):
            if 'SAL.xlsx' in file:
                df = df.append(pd.read_csv(dirpath+file))
        df.rename({'Cruise':'Cruise ID','Station':'Station #','Sample':'Sample ID','Salinity':'Salinity [psu]'},
          axis='columns',inplace=True)
        df.dropna(inplace=True)
        df['Station #'] = df['Station #'].apply(lambda x: str( int(x)).zfill(3))
        df['Niskin #'] = df['Niskin #'].apply(lambda x: str( int(x)))
        df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
    else:
        for file in os.listdir(dirpath):
            if 'SAL.csv' in file:
                df = df.append(pd.read_csv(dirpath+file))
        df.dropna(inplace=True)
        df['Station #'] = df['Station #'].apply(lambda x: str( int(x)).zfill(3))
        df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
        df.drop(columns=[x for x in list(df.columns.values) if 'unnamed' in x.lower()],inplace=True)

    # Save the processed summary file for salinity
    df.to_csv(dirpath+'SAL_Summary.csv')
    
    
def process_oxy_files(dirpath):
    df = pd.DataFrame()
    for file in os.listdir(dirpath):
        if 'OXY' in file:
            df = df.append(pd.read_excel(dirpath+file))
    # Rename and clean up the oxygen data to be uniform across data sets
    df.rename({'Cruise':'Cruise ID','Station':'Station #','Sample#':'Sample ID','Oxy':'Oxygen [mL/L]','Unit':'Units'},
          axis='columns',inplace=True)
    df.dropna(inplace=True)
    df['Station #'] = df['Station #'].apply(lambda x: str( int(x)).zfill(3))
    df['Niskin #'] = df['Niskin #'].apply(lambda x: str( int(x)))
    df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
    df['Cruise ID'] = df['Cruise ID'].apply(lambda x: x.replace('O','0'))
    
    # Save the processed summary file for oxygen
    df.to_csv(dirpath+'OXY_Summary.csv')

### CTD Sampling Log

In [249]:
os.listdir(basepath+array+cruise+water_dir)

['Pioneer-5_AT-31A_chlorophyll_analysis.xlsx',
 'PIONEER-5_AT-31A_CTD_Sample_Summary.csv',
 'PIONEER-5_AT-31A_CTD_sampling_logs.xlsx',
 'Pioneer-5_AT-31A_Nutrient_Lab_Results.xlsx',
 'Pioneer-5_AT-31B_chlorophyll_analysis.xlsx',
 'PIONEER-5_AT-31B_CTD_sampling_logs.xlsx',
 'Pioneer-5_AT-31B_Nutrient_Lab_Results.xlsx',
 'PIONEER-5_AT-31C_CTD_sampling_logs.xlsx',
 'Salts and O2']

In [250]:
summary_sheet_path = basepath+array+cruise+water_dir+'PIONEER-5_AT-31B_CTD_sampling_logs.xlsx'

In [251]:
sample_log = pd.read_excel(summary_sheet_path,sheet_name='Summary',header=0)

In [252]:
sample_log.head()

,Station-Cast #,Niskin #,Rosette Position,Date,Time,Trip Depth,Oxygen Bottle #,Ph Bottle #,DIC/TA Bottle #,Salts Bottle #,Nitrate Bottle 1,Chlorophyll Brown Bottle #,Chlorophyll Filter Sample # Cast #/Depth/Bottle #/,Chlorophyll Brown Bottle Volume,Chlorophyll LN Tube,Comments
0,1,1,1,2015-10-21,18:08:00,88.0,B1,691.0,692.0,T1,1-1,1.0,01/01,539.0,NaN,NaN
1,1,2,2,2015-10-21,18:08:00,88.0,B2,693.0,694.0,T2,1-2,2.0,01/02,539.0,NaN,Dupes
2,1,3,3,2015-10-21,18:11:00,73.0,B3,695.0,696.0,T3,1-3,3.0,01/03,539.0,NaN,Bottle 695 may not have received 100 ul mecuri...
3,1,4,4,2015-10-21,18:11:00,73.0,B4,NaN,NaN,T4,NaN,4.0,01/04,539.0,NaN,NaN
4,1,5,5,2015-10-21,18:15:00,33.0,B5,697.0,698.0,T5,1-4,5.0,01/05,539.0,NaN,NaN


In [253]:
# # Rename the column headers
# sample_log.rename(columns=lambda x: 'Sample Log: ' + x.strip(), inplace=True)
# sample_log['Sample Log: Niskin #'] = sample_log['Sample Log: Niskin #'].apply(lambda x: int(x.replace('*','')) if type(x) == str else x )
# sample_log['Sample Log: Niskin #'] = sample_log['Sample Log: Niskin #'].apply(lambda x: x if np.isnan(x) else str( int(x) ) )
# sample_log['Sample Log: Cast #'] = sample_log['Sample Log: Station-Cast #'].apply(lambda x: str(x).zfill(3))

### Nutrient & Chlorophyll Data

In [254]:
os.listdir(basepath+array+cruise+water_dir)

['Pioneer-5_AT-31A_chlorophyll_analysis.xlsx',
 'PIONEER-5_AT-31A_CTD_Sample_Summary.csv',
 'PIONEER-5_AT-31A_CTD_sampling_logs.xlsx',
 'Pioneer-5_AT-31A_Nutrient_Lab_Results.xlsx',
 'Pioneer-5_AT-31B_chlorophyll_analysis.xlsx',
 'PIONEER-5_AT-31B_CTD_sampling_logs.xlsx',
 'Pioneer-5_AT-31B_Nutrient_Lab_Results.xlsx',
 'PIONEER-5_AT-31C_CTD_sampling_logs.xlsx',
 'Salts and O2']

In [255]:
chlfile = 'Pioneer-5_AT-31B_chlorophyll_analysis.xlsx'
nutfile = 'Pioneer-5_AT-31B_Nutrient_Lab_Results.xlsx'

In [256]:
nutrients = pd.read_excel(basepath+array+cruise+water_dir+nutfile)

In [257]:
nutrients.head()

,Avg: Nitrate+Nitrite [µmol/L],Avg: Ammonium [µmol/L],Avg: Phosphate [µmol/L],Avg: Silicate [µmol/L],Avg: Nitrite [µmol/L],Avg: Nitrate [µmol/L]
1-1,7.16299,0.760683,0.539484,8.700655,<0.04,7.16299
1-2,6.89855,0.981945,0.539037,8.448336,<0.04,6.89855
1-3,5.0916,1.099319,0.393533,5.238539,<0.04,5.0916
1-4,0.42426,1.161752,0.165651,1.341621,<0.04,0.42426
1-5,2.14674,1.496392,0.119537,0.851766,<0.04,2.14674


In [258]:
chl = pd.read_excel(basepath+array+cruise+water_dir+chlfile)
chl.head()

,Cruise #:,Date,Station Start Time (UTC),Station End Time (UTC),Niskin Trip Time,Lat,Lon,Station Depth,Station-Cast #,Niskin #,...,Rb,Ra,blank,Rb-blank,Ra-blank,Chl (ug/l),Phaeo (ug/l),Cal_Date,Fluorometer,Comments
0,AT31-B,2015-10-21,18:03:00,NaN,18:08:00,40° 21.81' N,70° 53.001' W,96,1,1,...,84.38,61.04,0.851000,83.529000,60.189000,0.218483,0.275065,2015-12-09,NaN,NaN
1,AT31-B,2015-10-21,18:03:00,NaN,18:08:00,40° 21.81' N,70° 53.001' W,96,1,2,...,88.25,64.85,0.847833,87.402167,64.002167,0.219045,0.305772,2015-12-09,NaN,NaN
2,AT31-B,2015-10-21,18:03:00,NaN,18:11:00,40° 21.81' N,70° 53.001' W,96,1,3,...,75.56,51.62,0.847833,74.712167,50.772167,0.224100,0.192231,2015-12-09,NaN,NaN
3,AT31-B,2015-10-21,18:03:00,NaN,18:11:00,40° 21.81' N,70° 53.001' W,96,1,4,...,79.11,54.16,0.851000,78.259000,53.309000,0.233554,0.203579,2015-12-09,NaN,pick copepod
4,AT31-B,2015-10-21,18:03:00,NaN,18:15:00,40° 21.81' N,70° 53.001' W,96,1,5,...,302.50,185.20,0.847833,301.652167,184.352167,1.098032,0.413652,2015-12-09,NaN,NaN


In [259]:
os.listdir(basepath+array+cruise+water_dir+'Salts and O2')

['AT31-A', 'AT31-B', 'AT31-C']

In [260]:
# Load the Salinity and oxygen summaries
sal = pd.read_csv(basepath+array+cruise+water_dir+'Salts and O2/AT31-A/SAL_Summary.csv')
if 'case' in [x.lower() for x in sal.columns.values]:
    sal['Sample ID'] = sal['Case'] + sal['Sample ID'].apply(lambda x: str(x)) 
oxy = pd.read_csv(basepath+array+cruise+water_dir+'Salts and O2/AT31-A/OXY_Summary.csv')
if 'case' in [x.lower() for x in oxy.columns.values]:
    oxy['Sample ID'] = oxy['Case'] + oxy['Sample ID'].apply(lambda x: str(x)) 

In [261]:
sal.head()

,Unnamed: 0,Cruise ID,Station #,Cast #,Case,Sample ID,Salinity [psu]
0,0,SHORE,1,1,T,T1,35.0600
1,1,SHORE,1,1,T,T2,35.0587
2,2,SHORE,1,1,T,T3,35.1874
3,3,SHORE,1,1,T,T4,35.1874
4,4,SHORE,1,1,T,T5,35.7268


In [262]:
sample_log.head()

,Station-Cast #,Niskin #,Rosette Position,Date,Time,Trip Depth,Oxygen Bottle #,Ph Bottle #,DIC/TA Bottle #,Salts Bottle #,Nitrate Bottle 1,Chlorophyll Brown Bottle #,Chlorophyll Filter Sample # Cast #/Depth/Bottle #/,Chlorophyll Brown Bottle Volume,Chlorophyll LN Tube,Comments
0,1,1,1,2015-10-21,18:08:00,88.0,B1,691.0,692.0,T1,1-1,1.0,01/01,539.0,NaN,NaN
1,1,2,2,2015-10-21,18:08:00,88.0,B2,693.0,694.0,T2,1-2,2.0,01/02,539.0,NaN,Dupes
2,1,3,3,2015-10-21,18:11:00,73.0,B3,695.0,696.0,T3,1-3,3.0,01/03,539.0,NaN,Bottle 695 may not have received 100 ul mecuri...
3,1,4,4,2015-10-21,18:11:00,73.0,B4,NaN,NaN,T4,NaN,4.0,01/04,539.0,NaN,NaN
4,1,5,5,2015-10-21,18:15:00,33.0,B5,697.0,698.0,T5,1-4,5.0,01/05,539.0,NaN,NaN


In [263]:
# Now need to mak
sample_log = sample_log.merge(sal[['Station #','Sample ID','Salinity [psu]']], how='left', left_on=['Station-Cast #','Salts Bottle #'], right_on=['Station #','Sample ID'])

In [264]:
sample_log.rename({'Salinity [psu]':'Discrete Salinity [psu]'},axis='columns',inplace=True)
sample_log.drop(['Station #','Sample ID'],axis='columns',inplace=True)

In [265]:
sample_log.rename(columns=lambda x: x.strip(),inplace=True)

In [266]:
oxy.head()

,Unnamed: 0,Cruise ID,Station #,Niskin #,Case,Sample ID,Oxygen [mL/L],Units
0,1,AT31,1,1.0,B,B1,4.700,mL/L
1,2,AT31,1,2.0,B,B2,4.791,mL/L
2,3,AT31,1,3.0,B,B3,3.123,mL/L
3,4,AT31,1,4.0,B,B4,3.160,mL/L
4,5,AT31,1,5.0,B,B5,5.112,mL/L


In [267]:
sample_log = sample_log.merge(oxy[['Station #','Sample ID','Oxygen [mL/L]']], how='left', left_on=['Station-Cast #','Oxygen Bottle #'], right_on=['Station #','Sample ID'])

In [268]:
sample_log.rename({'Oxygen [mL/L]':'Discrete Oxygen [mL/L]'},axis='columns',inplace=True)
sample_log.drop(['Station #','Sample ID'],axis='columns',inplace=True)

In [269]:
nutrients.reset_index(inplace=True)

In [270]:
nutrients.rename({'index':'Sample ID'},axis='columns',inplace=True)

In [272]:
sample_log = sample_log.merge(nutrients, how='left', left_on=['Nitrate Bottle 1'], right_on=['Sample ID'])

In [273]:
sample_log.rename(columns=lambda x: x.replace('Avg:', 'Discrete'), inplace=True)
sample_log.drop(['Sample ID'],axis='columns',inplace=True)

In [274]:
sample_log.columns.values

array(['Station-Cast #', 'Niskin #', 'Rosette Position', 'Date', 'Time',
       'Trip Depth', 'Oxygen Bottle #', 'Ph Bottle #', 'DIC/TA Bottle #',
       'Salts Bottle #', 'Nitrate Bottle 1', 'Chlorophyll Brown Bottle #',
       'Chlorophyll Filter Sample # \nCast #/Depth/Bottle #/',
       'Chlorophyll Brown Bottle Volume', 'Chlorophyll LN Tube',
       'Comments', 'Discrete Salinity [psu]', 'Discrete Oxygen [mL/L]',
       'Discrete Nitrate+Nitrite [µmol/L]', 'Discrete Ammonium [µmol/L]',
       'Discrete Phosphate [µmol/L]', 'Discrete Silicate [µmol/L]',
       'Discrete Nitrite [µmol/L]', 'Discrete Nitrate [µmol/L]'],
      dtype=object)

In [275]:
# Now add the chlorophyll data
chl.columns.values

array(['Cruise #:', 'Date', 'Station \nStart Time (UTC)',
       'Station \nEnd Time (UTC)', 'Niskin Trip Time', 'Lat', 'Lon',
       'Station Depth', 'Station-Cast #', 'Niskin #', 'Trip \nDepth',
       'Brown Bottle #', 'Replicate', 'Water Depth Rep',
       'Filter \nSample #', 'Vol\nFilt', 'Filter\nSize', 'Vol Extracted',
       'Sample', '90% Acetone', 'Dilution During Reading',
       'Chl_Cal_Filename', 'tau_Calibration', 'Fd_Calibration', 'Rb',
       'Ra', 'blank', 'Rb-blank', 'Ra-blank', 'Chl (ug/l)',
       'Phaeo (ug/l)', 'Cal_Date', 'Fluorometer', 'Comments'],
      dtype=object)

In [276]:
chl_df = chl[['Station-Cast #','Brown Bottle #','Chl (ug/l)','Phaeo (ug/l)']]
chl_df.rename(columns=lambda x: 'Discrete ' + x, inplace=True)
#chl_df.rename({'Discrete quality_flag':'Discrete Chl quality flag'},axis='columns',inplace=True)

In [277]:
sample_log = sample_log.merge(chl_df, how='left', left_on=['Station-Cast #','Chlorophyll Brown Bottle #'], right_on=['Discrete Station-Cast #','Discrete Brown Bottle #'])
sample_log.drop(['Discrete Station-Cast #','Discrete Brown Bottle #'],axis='columns',inplace=True)

In [278]:
# Now load the CTD summary data
CTD = pd.read_csv(basepath+array+cruise+'Leg_1/'+ctd_dir+'CTD_Summary.csv')

In [279]:
column_list = []
for name in list(sample_log.columns.values):
    if 'Discrete' in name:
        column_list.append(name)
column_list.append('Station-Cast #')
column_list.append('Niskin #')

In [280]:
discrete_data = sample_log[column_list]

In [281]:
discrete_data

,Discrete Salinity [psu],Discrete Oxygen [mL/L],Discrete Nitrate+Nitrite [µmol/L],Discrete Ammonium [µmol/L],Discrete Phosphate [µmol/L],Discrete Silicate [µmol/L],Discrete Nitrite [µmol/L],Discrete Nitrate [µmol/L],Discrete Chl (ug/l),Discrete Phaeo (ug/l),Station-Cast #,Niskin #
0,35.0600,4.700,7.16299,0.760683,0.539484,8.700655,<0.04,7.16299,0.218483,0.275065,1,1
1,35.0587,4.791,6.89855,0.981945,0.539037,8.448336,<0.04,6.89855,0.219045,0.305772,1,2
2,35.1874,3.123,5.0916,1.099319,0.393533,5.238539,<0.04,5.0916,0.224100,0.192231,1,3
3,35.1874,3.160,NaN,NaN,NaN,NaN,NaN,NaN,0.233554,0.203579,1,4
4,35.7268,5.112,0.42426,1.161752,0.165651,1.341621,<0.04,0.42426,1.098032,0.413652,1,5
5,35.7262,5.170,2.14674,1.496392,0.119537,0.851766,<0.04,2.14674,1.004423,0.412115,1,6
6,35.8341,5.098,<0.04,1.131784,0.132521,0.867568,<0.04,<0.04,1.627343,0.487112,1,7
7,35.8346,5.110,NaN,NaN,NaN,NaN,NaN,NaN,1.554056,0.369399,1,8
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2


In [282]:
CTD = CTD.merge(discrete_data, how='left', left_on=['Cast #','Niskin #'], right_on=['Station-Cast #','Niskin #'])

In [283]:
CTD.head()

,Unnamed: 0,Niskin #,Date Time,"Pressure, Digiquartz [db]","Depth [salt water, m]",Latitude [deg],Longitude [deg],"Temperature [ITS-90, deg C]","Temperature, 2 [ITS-90, deg C]",Conductivity [S/m],...,Discrete Oxygen [mL/L],Discrete Nitrate+Nitrite [µmol/L],Discrete Ammonium [µmol/L],Discrete Phosphate [µmol/L],Discrete Silicate [µmol/L],Discrete Nitrite [µmol/L],Discrete Nitrate [µmol/L],Discrete Chl (ug/l),Discrete Phaeo (ug/l),Station-Cast #
0,0,1,Oct 12 2015 16:39:05,443.326,439.460,39.94014,-70.8834,6.0237,6.0239,3.463375,...,4.700,7.16299,0.760683,0.539484,8.700655,<0.04,7.16299,0.218483,0.275065,1.0
1,1,2,Oct 12 2015 16:39:23,443.278,439.413,39.94014,-70.8834,6.0226,6.0234,3.463267,...,4.791,6.89855,0.981945,0.539037,8.448336,<0.04,6.89855,0.219045,0.305772,1.0
2,2,3,Oct 12 2015 16:45:09,251.878,249.798,39.94014,-70.8834,9.3260,9.3490,3.774700,...,3.123,5.0916,1.099319,0.393533,5.238539,<0.04,5.0916,0.224100,0.192231,1.0
3,3,4,Oct 12 2015 16:45:17,252.137,250.054,39.94014,-70.8834,9.3717,9.3705,3.779986,...,3.160,NaN,NaN,NaN,NaN,NaN,NaN,0.233554,0.203579,1.0
4,4,5,Oct 12 2015 16:55:13,52.160,51.755,39.94014,-70.8834,21.8404,21.8252,5.073489,...,5.112,0.42426,1.161752,0.165651,1.341621,<0.04,0.42426,1.098032,0.413652,1.0


In [284]:
CTD.drop(['Unnamed: 0'],axis='columns',inplace=True)

In [285]:
CTD.fillna(-9999999, inplace=True)
CTD['Date Time'] = CTD['Date Time'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%dT%H:%M:%SZ'))

In [286]:
CTD.head()

,Niskin #,Date Time,"Pressure, Digiquartz [db]","Depth [salt water, m]",Latitude [deg],Longitude [deg],"Temperature [ITS-90, deg C]","Temperature, 2 [ITS-90, deg C]",Conductivity [S/m],"Conductivity, 2 [S/m]",...,Discrete Oxygen [mL/L],Discrete Nitrate+Nitrite [µmol/L],Discrete Ammonium [µmol/L],Discrete Phosphate [µmol/L],Discrete Silicate [µmol/L],Discrete Nitrite [µmol/L],Discrete Nitrate [µmol/L],Discrete Chl (ug/l),Discrete Phaeo (ug/l),Station-Cast #
0,1,2015-10-12T16:39:05Z,443.326,439.460,39.94014,-70.8834,6.0237,6.0239,3.463375,3.463428,...,4.700,7.16299,7.606827e-01,5.394843e-01,8.700655e+00,<0.04,7.16299,0.218483,0.275065,1.0
1,2,2015-10-12T16:39:23Z,443.278,439.413,39.94014,-70.8834,6.0226,6.0234,3.463267,3.463380,...,4.791,6.89855,9.819450e-01,5.390366e-01,8.448336e+00,<0.04,6.89855,0.219045,0.305772,1.0
2,3,2015-10-12T16:45:09Z,251.878,249.798,39.94014,-70.8834,9.3260,9.3490,3.774700,3.777214,...,3.123,5.0916,1.099319e+00,3.935325e-01,5.238539e+00,<0.04,5.0916,0.224100,0.192231,1.0
3,4,2015-10-12T16:45:17Z,252.137,250.054,39.94014,-70.8834,9.3717,9.3705,3.779986,3.779783,...,3.160,-9999999,-9.999999e+06,-9.999999e+06,-9.999999e+06,-9999999,-9999999,0.233554,0.203579,1.0
4,5,2015-10-12T16:55:13Z,52.160,51.755,39.94014,-70.8834,21.8404,21.8252,5.073489,5.071585,...,5.112,0.42426,1.161752e+00,1.656508e-01,1.341621e+00,<0.04,0.42426,1.098032,0.413652,1.0


In [208]:
summary_sheet_path

'/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Pioneer/Pioneer-05/Water Sampling/PIONEER-5_AT-31A_CTD_sampling_logs.xlsx'

In [215]:
summary_name = summary_sheet_path.replace('sampling_logs.xlsx','Sample_Summary.csv')
summary_name

'/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Pioneer/Pioneer-05/Water Sampling/PIONEER-5_AT-31A_CTD_Sample_Summary.csv'

In [216]:
CTD.to_csv(summary_name)

In [ ]:
nutrient_bottles = sample_log['Sample Log: Nitrate Bottle 1'].str.split(',').apply(pd.Series, 1).stack()
nutrient_bottles.index = nutrient_bottles.index.droplevel(-1)
nutrient_bottles.name = 'Nitrate Bottle #'

In [ ]:
nutrient_bottles

In [ ]:
# Add the nutrient bottle number back into the sample log, and remove the excess '.'
sample_log = sample_log.join(nutrient_bottles)
sample_log['Nitrate Bottle #'] = sample_log['Nitrate Bottle #'].apply(lambda x: x.replace('.','') if type(x) == str else x)
sample_log['Nitrate Bottle #'] = sample_log['Nitrate Bottle #'].apply(lambda x: x.replace(' ','') if type(x) == str else x)

In [ ]:
# Now I can add the nutrient bottle data to the sample log before loading
sample_log = sample_log.merge(nutrients, how='left', left_on='Nitrate Bottle #', right_on='Sample ID')

In [ ]:
sample_log.columns.values

In [ ]:
sample_columns = ['Sample Log: Cast #','Sample Log: Niskin #','SAL: Salinity','OXY: Oxy','Avg: Nitrate+Nitrite [µmol/L]', 'Avg: Ammonium [µmol/L]',
    'Avg: Phosphate [µmol/L]', 'Avg: Silicate [µmol/L]', 'Avg: Nitrite [µmol/L]', 'Avg: Nitrate [µmol/L]']

In [ ]:
result = df.merge(sample_log[sample_columns], how='left',
                  left_on=['Cast #','Niskin #'], right_on=['Sample Log: Cast #','Sample Log: Niskin #'])

In [ ]:
result

In [ ]:
result.columns.values

In [ ]:
sal_unit = '[' + list(set(sal_df['SAL: Unit']) )[0] + ']'
oxy_unit = '[' + list(set(oxy_df['OXY: Unit']) )[0] + ']'
sal_unit, oxy_unit

In [ ]:
result.drop(['Sample Log: Cast #','Sample Log: Niskin #'], axis=1, inplace=True)

In [ ]:
result.rename(columns=lambda x: x.replace('SAL:','Bottle') + ' ' + sal_unit if 'SAL:' in x else x, inplace=True)
result.rename(columns=lambda x: x.replace('OXY:','Bottle') + ' ' + oxy_unit if 'OXY:' in x else x, inplace=True)
result.rename(columns=lambda x: x.replace('Avg:','Bottle') if 'Avg:' in x else x, inplace=True)

In [ ]:
result

In [ ]:
# Replace all of the nans with -999999 and save to a csv
result.fillna(str(-9999999),inplace=True)

In [ ]:
result

In [ ]:
result.to_csv(salts_and_o2_path+'Irminger-3_Summary.csv')

In [ ]:
os.getcwd()